In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


file_path = "Online Retail.xlsx"
data = pd.read_excel(file_path)

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
data.shape

(541909, 8)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Load the dataset (assuming it's already in the `online_retail_data` DataFrame)
# Assuming the data has been loaded as online_retail_data from the Excel file

# Focus on useful columns
useful_columns = [
    "InvoiceNo",      # Transaction identifier
    "StockCode",      # Product code
    "Description",    # Product description (can be ignored if not needed for modeling)
    "Quantity",       # Number of items
    "InvoiceDate",    # Date of purchase
    "UnitPrice",      # Price per unit
    "CustomerID",     # Customer identifier
    "Country",        # Customer's country
]

# Filter the data
filtered_data = data[useful_columns]

# 1. Handle missing values
filtered_data.dropna(subset=["CustomerID"], inplace=True)  # Drop rows without a customer
filtered_data.dropna(subset=["Quantity", "UnitPrice"], how='any', inplace=True)  # Drop rows with missing Quantity or UnitPrice

# 2. Create TotalSales
filtered_data["TotalSales"] = filtered_data["Quantity"] * filtered_data["UnitPrice"]

# 3. Extract time-based features from InvoiceDate
filtered_data["InvoiceDate"] = pd.to_datetime(filtered_data["InvoiceDate"])
filtered_data["InvoiceYear"] = filtered_data["InvoiceDate"].dt.year
filtered_data["InvoiceMonth"] = filtered_data["InvoiceDate"].dt.month
filtered_data["InvoiceDay"] = filtered_data["InvoiceDate"].dt.day
filtered_data["InvoiceHour"] = filtered_data["InvoiceDate"].dt.hour
filtered_data["Weekday"] = filtered_data["InvoiceDate"].dt.weekday  # 0=Monday, 6=Sunday

# Drop the original InvoiceDate column as it's no longer needed
filtered_data.drop(columns=["InvoiceDate"], inplace=True)

# 4. Encode Categorical Columns
# One-hot encode 'Country' and 'StockCode'

encoder = OneHotEncoder(drop='first', sparse_output=False)  # Updated for newer scikit-learn versions
encoded_country = encoder.fit_transform(filtered_data[["Country"]])

# Convert the encoded output to a DataFrame
encoded_country_df = pd.DataFrame(encoded_country, columns=encoder.get_feature_names_out(["Country"]))

# Concatenate the encoded columns back to the dataframe
filtered_data = pd.concat([filtered_data, encoded_country_df], axis=1)

# Concatenate the encoded columns back to the dataframe
filtered_data = pd.concat([filtered_data, encoded_country_df], axis=1)

# 5. Scale Continuous Columns (MinMax Scaling for Quantity, UnitPrice, TotalSales)
scaler = MinMaxScaler()
continuous_columns = ["Quantity", "UnitPrice", "TotalSales", "InvoiceYear", "InvoiceMonth", "InvoiceDay", "InvoiceHour", "Weekday"]
filtered_data[continuous_columns] = scaler.fit_transform(filtered_data[continuous_columns])

# Display the cleaned and preprocessed dataset
filtered_data.head()


<ipython-input-57-7ffde11fdb31>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=["CustomerID"], inplace=True)  # Drop rows without a customer
<ipython-input-57-7ffde11fdb31>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=["Quantity", "UnitPrice"], how='any', inplace=True)  # Drop rows with missing Quantity or UnitPrice
<ipython-input-57-7ffde11fdb31>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,TotalSales,InvoiceYear,InvoiceMonth,...,Country_RSA,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,0.500037,0.000065,17850.0,United Kingdom,0.500045,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,536365,71053,WHITE METAL LANTERN,0.500037,0.000087,17850.0,United Kingdom,0.500060,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,0.500049,0.000071,17850.0,United Kingdom,0.500065,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,0.500037,0.000087,17850.0,United Kingdom,0.500060,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,0.500037,0.000087,17850.0,United Kingdom,0.500060,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
X = filtered_data.drop(columns=["TotalSales", "InvoiceNo", "Description", "StockCode"])
y = filtered_data["TotalSales"]  # Target: TotalSales

# Check for any non-numeric columns in X
non_numeric_columns = X.select_dtypes(include=["object"]).columns
print("Non-numeric columns before encoding:", non_numeric_columns)

# Encoding categorical columns (e.g., 'Country' and 'StockCode') if not already done
X = pd.get_dummies(X, drop_first=True)  # One-hot encode categorical features

# Check if there are still non-numeric columns
non_numeric_columns = X.select_dtypes(include=["object"]).columns
print("Non-numeric columns after encoding:", non_numeric_columns)

# Train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (important for ANN performance)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Only numeric data
X_test_scaled = scaler.transform(X_test)        # Only numeric data


Non-numeric columns before encoding: Index(['Country'], dtype='object')
Non-numeric columns after encoding: Index([], dtype='object')


In [ ]:
print("Missing values in features (X):", X.isnull().sum())
print("Missing values in target (y):", y.isnull().sum())

# Drop rows where any of the features or target is missing
X = X.dropna()
y = y[X.index]  # Make sure to drop corresponding rows in y

# Check the shapes after dropping missing data
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Now split the data again after ensuring no missing values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Missing values in features (X): Quantity                        104134
UnitPrice                       104134
CustomerID                      104134
InvoiceYear                     104134
InvoiceMonth                    104134
                                 ...  
Country_Switzerland                  0
Country_USA                          0
Country_United Arab Emirates         0
Country_United Kingdom               0
Country_Unspecified                  0
Length: 116, dtype: int64
Missing values in target (y): 104134
Shape of X: (302695, 116)
Shape of y: (302695,)


In [ ]:
# Scale the data (important for ANN performance)


# Define the ANN model
model = Sequential([
    # Input Layer
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.3),  # Dropout to prevent overfitting

    # Hidden Layers
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),

    # Output Layer
    Dense(1, activation='linear')  # Output a single continuous value for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                      │ (None, 128)                 │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,345 (99.00 KB)

 Trainable params: 25,345 (99.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 154.1300 - mae: 1.2967 - val_loss: 6403.2539 - val_mae: 1.6736
Epoch 2/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 381.1430 - mae: 1.4192 - val_loss: 6342.1079 - val_mae: 1.6233
Epoch 3/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 403.7413 - mae: 1.4633 - val_loss: 6196.9482 - val_mae: 1.6123
Epoch 4/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 451.2156 - mae: 1.4637 - val_loss: 5612.4336 - val_mae: 1.5730
Epoch 5/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 313.5252 - mae: 1.4864 - val_loss: 4858.6011 - val_mae: 1.5552
Epoch 6/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 232.8618 - mae: 1.4251 - val_loss: 3784.4780 - val_mae: 1.5655
Epoch 7/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 329.6107 - mae: 1.4591 - val_loss: 2648.8354 - val_mae: 1.5473
Epoch 8/50
7568/7568 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - loss: 113.2883 - mae: 1.2894 - val_loss: 1732.6698 - val_mae: